# Miking

Miking (Meta vIKING) is a meta language system for creating embedded domain-specific and general-purpose languages. Miking is not a programming language, but rather a language system for
creating languages and generating efficient compilers.

## This Notebook

This notebook gives a comprehensive introduction to the Miking language system in an interactive format. It can be seen as a complement to the main `README` intended to be easier to get started with but not covering the same depth.

When going through the notebook, we recommend playing around with it by changing the values in code cells or trying out your own examples.

## Table of Contents
1. [MCore](#MCore)
2. [Python Intrinsics](#Python-Intrinsics)
3. [Additional Notebook Features](#Notebook-Features)

## MCore

MCore (Miking Core) is the core language of the Miking system. It is
based on a typed Lambda Calculus (Note: the type system is under
development, and the current implementation is untyped).

MCore consists of two parts:

* **MExpr** is an MCore expression. A Miking language is always translated into an MExpr, before it is further evaluated or compiled into machine code.

* **MLang** which is a language for defining and composing language fragments. MLang is formally translated into an MExpr.

In Jupyter, we support both MExpr and MLang in code cells. In the following sections, we will use both without worrying too much about differentiating between the two.

### Values and Function Applications

MExpr features a number of different types of values:

- Bool
- Int
- Char
- Float
- String
- Sequence
- Unit
- Record
- Tuple
- Function
- User-defined

There are also a number of builtin functions that can be used. For more information on these, see [README.md](./README.md)
For instance, the following expression will add the two values 2 and 3.

In [ ]:
addi 2 3

Next, let's print the iconic `Hello, world!` message

In [ ]:
print "Hello, world!"

### Let Expressions

Expressions can be given names using `let` expressions. For instance

In [ ]:
let x = addi 1 2 in
eqi x 3

introduces a new name `x`. The built-in function `addi`, as we have seen, performs an addition between two integers. Note that MCore uses a call-by-value evaluation order, which means that expressions are evaluated into a value before they are applied to a function or substituted using a `let` expression. Hence, the expression `addi 1 2` is evaluated before it is substituted for `x` in the rest of the expression.

Finally, the `eqi` builtin function is used to check that `x` is equal to 3.

### Top-level Definitions

In the above example, `x` is defined only in the scope of the expression following the `in` keyword. To define a variable in the top-level scope, we can simply omit the `in` keyword:

In [ ]:
let x = addi 1 2

This lets us reuse `x` in other code cells:

In [ ]:
addi x 10

If you want to evaluate an expression in the same cell as a definition, you need to take some care. Simply putting one after the other won't produce the correct result:

In [ ]:
let y = addi x 10

y

Instead, either use a let binding with the `in` keyword, or use the `mexpr` keyword to separate the expression to evaluate, like this

In [ ]:
let y = addi x 10

mexpr
y

Now, both `x` and `y` are in the global scope.

### Functions

Functions are always defined anonymously as lambda functions. If you would like to give a function a name, a `let` expression can be used. For instance, the following program defines a function `double` that doubles the value of its argument.

In [ ]:
let double = lam x. muli x 2

In [ ]:
double 5

Types can be expressed in MCore programs, but they are currently not checked. For instance, the `double` function can be written as

```
let double = lam x:Int. muli x 2 in
```

This means that `double` has type `Int -> Int`, which can also be expressed as part of the `let` expression.

```
let double : Int -> Int = lam x. muli x 2 in
```

A function with several parameters are expressed using currying, using nested lambda expressions. For instance,

In [ ]:
let foo = lam x. lam y. addi x y

creates a function `foo` that takes two arguments.

In [ ]:
foo 2 3

### `if` Expressions

Conditional expressions can be expressed using `if` expressions. The program

In [ ]:
let x = 5 in
let answer = if (lti x 10) then "yes" else "no" in
answer

checks if `x` is less than 10 (using the `lti` function with signature `Int -> Int -> Bool`). If it is true, the string `"yes"` is returned, else string `"no"` is returned.

### Recursion

A normal `let` expression cannot be used to define recursive functions. Instead, recursion can be defined using *recursive lets*, starting with the `recursive` keyword:

In [ ]:
recursive
let fact = lam n.
  if eqi n 0
    then 1
    else muli n (fact (subi n 1))
in
fact 4

It is also possible to use a recursive let in the top-level:

In [ ]:
recursive
let fact = lam n.
  if eqi n 0
    then 1
    else muli n (fact (subi n 1))
end

In [ ]:
fact 4

Recursive lets can also be used to define mutually recursive functions. For instance:

In [ ]:
recursive
let odd = lam n.
    if eqi n 1 then true
    else if lti n 1 then false
    else even (subi n 1)
let even = lam n.
    if eqi n 0 then true
    else if lti n 0 then false
    else odd (subi n 1)
end

In [ ]:
odd 4

In [ ]:
even 4

### Tuples

Product types can be expressed using tuples. An n-tuple is defined using syntax `(e_1, ..., e_n)` where `e_1` to `e_n` are MCore expressions. Extracting a value from a tuple (projection) is performed using an expression `e.n` where `e` is the expression that is evaluated into a tuple, and `n` is an integer number representing the index of an element in the tuple. The first index in a tuple is `0`.

For instance, in the following code

In [ ]:
let t = (addi 1 2, "hi", 80)

In [ ]:
t.0

In [ ]:
t.1

In [ ]:
t.2

we create a 3-tuple `t` and project out its first value. Note that the different elements of a tuple can have different types. In this case, tuple `t` has type `(Int, String, Int)`.

Singleton tuples are also supported: `(x,)` represents a tuple with `x` as its
only element.

### Records

Another more general form of product types are records. A record has
named fields that can have different types. For instance,

In [ ]:
{name = "foobar", age = 42}

defines a record of type `{name : string, age : int}`. The order of the fields does not matter: the following would define the same record.

In [ ]:
{age = 42, name = "foobar"}

To project out a value, a dot notation may be used.

In [ ]:
let r1 = {age = 42, name = "foobar"} in
r1.age

A record type is not just a general product type in MCore, it is the only
product type. That is, a tuple is just *syntactic sugar* for a record. This means that the compiler encodes a tuple as a record, where the names of the fields are numbers `0`, `1`, etc. Labels can internally be any kind of string. For strings that cannot be defined as a normal identifier, the label form `#label"x"`
can be used, where `x` is the string of the label.

The following example shows how a tuple is actually encoded as a
record.

In [ ]:
{#label"0" = true, #label"1" = 5}

### Data Types and `match` expressions

Sum types or variant types are constructed using `con` expressions (constructor expressions). In contrast to most other functional languages, the introduction of a new data type and the introduction of constructors are separated. For instance,

In [ ]:
type Tree
con Node : (Tree,Tree) -> Tree
con Leaf : (Int) -> Tree

introduces a new data type `Tree` and defines two new constructors `Node` and `Leaf`. Constructor `Leaf` takes just one argument (an integer value for the leaf) and returns a tree, whereas the `Node` constructor takes a tuple with two trees as input and constructs a new tree node.

For instance,

In [ ]:
let tree = Node(Node(Leaf 4, Leaf 2), Leaf 3)

is a small tree named `tree`.

Assume now that we want to count the sum of the values of all leaves in a tree. We can then write a recursive function that performs the counting:

In [ ]:
recursive
  let count = lam tree.
    match tree with Node t then
      let left = t.0 in
      let right = t.1 in
      addi (count left) (count right)
    else match tree with Leaf v then v
    else error "Unknown node"
in

count tree

The `match` expression inside the `count` function *deconstructs* data values by matching against a given constructor. For instance, the `match` expression

```
match tree with Node t then expr1 else expr2
```

matches the value after evaluating expression `tree` and checks if its outer most constructor is a `Node` constructor. If that is the case, the identifier `t` in expression `expr1` is bound to the tuple consisting of the node's two sub trees (recall the definition of the constructor `Node`).

### Pattern matching

In the previous match example, the `match` construct matched against
the constructor, but not against the actual data content. MExpr is
designed to be simple with few language construct, at the right level
of abstraction. If the abstraction level is too low, it is hard to
perform useful static analysis and code generation. As a consequence,
MExpr support *patterns* in `match` expressions. The `count` function
can be rewritten as


In [ ]:
recursive
  let count = lam tree.
    match tree with Node(left,right) then
      addi (count left) (count right)
    else match tree with Leaf v then v
    else error "Unknown node"
end

where the match construct matches against pattern `Node(left,right)`,
where `left` and `right` are pattern variables.

Remember, however, that tuples are just syntactic sugar for records. Hence, match line

```
    match tree with Node(left,right) then
```
is equivalent to the following
```
    match tree with Node {#label"0"=left,#label"1"=right} then
```
where the pattern is a *record pattern*.

Pattern matching is the general form of deconstructing data in MExpr. Patterns can also be nested:

In [ ]:
match {foo=7,bar={more="hello"}} with {foo=_,bar={more=str}} then str else ""

Note also the use of *wildcard* patterns `_` (used in the `foo`
field), which matches any value.

Finally, MExpr also supports more advanced patterns, including `AND` patterns (using infix notation `&`)

In [ ]:
match (1, 2) with (a, _) & b then (a, b) else (0, (0, 0))

`OR` patterns (using infix notation `|`)

In [ ]:
type K in
con K1: (Int) -> K in
con K2: (Int) -> K in

match K1 1 with K1 a | K2 a then a else 0

and `NOT` patterns (using the prefix notation `!`)

In [ ]:
match Some true with a & !(None ()) then a else Some false

In [ ]:
match None () with a & !(None ()) then a else Some false

These are present to make it possible to translate order-dependent patterns to order-*in*dependent patterns. For example, in OCaml,

```ocaml
match (opt1, opt2) with
| (Some a, _) -> a
| (_, Some a) -> a
| (_, _) -> 1
```

is order-dependent; any change in pattern order changes which match-arm is executed. To express this in an order-independent manner we `&` every pattern with the inverse (`!`) of the union (`|`) of the previous patterns. If we pretent for a moment that OCaml supports `&` and `!` in patterns they could then be written as:

```ocaml
match (opt1, opt2) with
| (Some a, _) -> a
| (_, Some a) & !(Some a, _) -> a
| (_, _) & !((Some a, _) | (_, Some a))-> 1
```

The order can now be changed freely without affecting the semantics. In practice `&` and `!` will probably rarely be used in manually written code, while `|` is rather more useful.


### Sequences

An MCore sequence is constructed using syntax `[e_1, ..., e_n]`. All elements in a sequence must have the same type. For instance, an expression

In [ ]:
[1,3,6,7,22,3]

has type `[Int]` whereas the expression

In [ ]:
["this", "is", "a", "test"]

has type `[String]`.

As you may already have noticed, a string itself is actually a sequence of characters. This means that the type `String` is just an abbreviation for the sequence type `[Char]`.

There are several operations defined for sequences, for instance, the `concat` function concatenates two sequences

In [ ]:
concat [1,3,5] [7,9]

or the `get` function picks out the nth element of a sequence

In [ ]:
get [3,5,8,9] 2

It is also possible to pattern match on sequences, to either extract the *tail* of a sequence, if the first part matches

In [ ]:
match "foobar" with "fo" ++ rest then rest else ""

or the *head* of a sequence if the last part matches:

In [ ]:
match "foobar" with first ++ "bar" then first else ""

It is even possible to extract the middle of a sequence, if the head and the tail matches:

In [ ]:
match "foobar" with "fo" ++ mid ++ "ar" then mid else ""

Again, matching can be combined and nested:

In [ ]:
match (1,[["a","b"],["c"]],76) with (1,b++[["c"]],76) then b else []

### Includes

To include code from the standard library, or from an MCore file you've written yourself, you can use the `include` keyword. This will introduce all top-level definitions from the included file into the top-level scope. The filepath to `include` can be any standard library file or a valid filepath to a user-defined file.

In [ ]:
include "string.mc"

Now, the functions from the standard library file `string.mc` can be used.

In [ ]:
print (strJoin ", " ["Hello", "world!"])

### Language Fragments

A language fragment contains definitions of (abstract) syntax, and
semantics ("interpreters") for that fragment. Any number of
language fragments can be defined in the top-level of an
MCore program. For example, here is a language fragment for simple
arithmetics:

In [ ]:
lang Arith
  syn Expr =
  | Num Int
  | Add (Expr, Expr)

  sem eval =
  | Num n -> Num n
  | Add (e1,e2) ->
    match eval e1 with Num n1 then
      match eval e2 with Num n2 then
        Num (addi n1 n2)
      else error "Not a number"
    else error "Not a number"
end

The fragment defines a syntactic form with two cases called
`Expr`, and an interpreter called `eval`. An interpreter in MLang
is a function that is always defined by cases over its last
argument (here, `eval` takes only a single argument). The body of
a case is a regular MExpr term, which has access to the name of
the value (if any) carried by the current syntactic form.

In an expression, a language fragment can be opened by
a `use` expression:

In [ ]:
use Arith in
eval (Add (Num 2, Num 3))

A `use` is translated into a series of MExpr definitions that
match the syntax and semantics of the specified language fragment.

An important feature of language fragments is that they can be
composed to form new language fragments. As an example, we might
want to extend our arithmetics language with booleans and `if`
expressions:

In [ ]:
lang Bool
  syn Expr =
  | True()
  | False()
  | If (Expr, Expr, Expr)

  sem eval =
  | True() -> True()
  | False() -> False()
  | If(cnd,thn,els) ->
    let cndVal = eval cnd in
    match cndVal with True() then eval thn
    else match cndVal with False() then eval els
    else error "Not a boolean"
end

lang ArithBool = Arith + Bool

In [ ]:
use ArithBool in
eval (Add (If (False(), Num 0, Num 5), Num 2))

The language fragment `ArithBool` is indistinguishable from a
language fragment with all the syntactic and semantic cases of
`Arith` and `Bool` merged. If we wanted, we could have added new
cases to the language composition as well, and refer to the syntax
and semantics of the fragments being composed:

In [ ]:
lang ArithBool = Arith + Bool
  syn Expr =
  | IsZero Expr

  sem eval =
  | IsZero e ->
    match eval e with Num n then
      if eqi n 0 then True() else False()
    else
      error "Not a number"
end

In [ ]:
use ArithBool in
eval (IsZero (If (False(), Num 1, Num 0)))

## Python Intrinsics

An optional feature of MCore is Python intrinsics, which allow calling Python functions from MCore code. The Jupyter kernel includes these features.

The following example shows how to use the intrinsics to sort a sequence using
Python's builtin `sorted` function. Before you can call a Python function, you will need to import the relevant Python module using `pyimport`.

In [ ]:
let builtins = pyimport "builtins"

Any module in the Python path can be imported in this way. Now, `pycall` can be used to call a function from that module.

In [ ]:
let x = [5,4,2,1,3]
let y = pycall builtins "sorted" (x,)

In [ ]:
y

The result of the `pycall` is a Python value. Python values can be passed to other Python functions, but not regular MCore functions:

In [ ]:
pycall builtins "print" (y,)

In [ ]:
map (addi 2) y

To recover an MCore value from a Python value, use the `pyconvert` intrinsic.

In [ ]:
let y_mcore = pyconvert y in
map (addi 2) y_mcore

Most basic values can be converted between Python and MCore types; the main exceptions are Python classes and MCore user-defined data types. For a detailed explanation, see [the main README](./README.md).

## Additional Notebook Features

In addition to what we've already seen, the Jupyter kernel also offers some additional features.

### Autocompletion
One thing that you may not have noticed is that autocompletion is available. To get completions, use `Tab` after starting to type a name. Try it out in the cell below.

In [ ]:
option

### Python Cells

The MCore kernel also allows executing Python code and interacting with it from
MCore. Use the special directive `%%python` at the top of a cell to evaluate
Python code.

For example, the following cell defines a Python function `foo` and calls it.

In [ ]:
%%python
def foo(x):
  print("foo"+x)

foo("bar")

You can call the functions you have defined in Python cells in normal MCore
cells by using the Python intrinsics. A user-defined function can
be called by importing and using the Python module `__main__`. For example,
consider the following cell:

In [ ]:
let m = pyimport "__main__" in
let x = "baz" in
pycall m "foo" (x,)

This cell calls the Python function `foo` defined above, printing
`foobaz` as expected.

### Plotting Graphs
It is possible to plot graphs using the Python library `matplotlib`.
The Jupyter kernel offers integration with `matplotlib` to display plots
directly in a notebook.

To use this functionality, first make sure that `matplotlib` is installed (if
not, you can install it using `pip`). Now, when you use `matplotlib`'s plot
functions in a notebook cell, the plots will be displayed as part of the cell's
output. For example, try the following cell:

In [ ]:
let plt = pyimport "matplotlib.pyplot"
let x = [1,2,4,8]
let _ = pycall plt "plot" (x,)

While this example uses the Python intrinsics, running the plot code directly in
a Python cell would of course also work.